In [211]:
import numpy as np
import xtools as xt
import matplotlib.pyplot as plt
import dimod
from openjij import SASampler, SQASampler

from atm.flight.generator import ScenarioGenerator, Scenario
from atm.optimizer.objective import *
from atm.separation import recat


In [430]:
def get_default_config():
    return xt.Config(dict(
        seed=0,
        num_runway=2,
        num_vol=30,
        penalty=100,
        scenario=dict(
            interval=60,
            window=120,
            mode="mix",
            standard="recat"
        )
    ))


In [13]:
def get_obj_multiple_runway(scenario: Scenario, separation: Separation, penalty_coef: float = 1.0) -> Callable:
    def func(xs):
        times = calc_assign_time_for_multi_runway(xs, scenario, separation)
        delays = calc_delay_for_multi_runway(xs, times, scenario)
        dues = get_due_for_multi_runway(xs, scenario)
        is_overtimes = check_over_time_for_multi_runway(times, dues)
        num_overtime = count_num_overtime(is_overtimes)

        return np.mean(np.concatenate(delays)).astype(float) + penalty_coef * num_overtime
    return func

In [35]:
def decode(x):
    return [
        x[0, x[1,] == q]
        for q in range(np.max(x[1,]) + 1)
    ]

In [278]:
def make_scenario(cf, separation_manner: Separation = recat.TBS, seed: int = None):
    if seed is not None:
        np.random.seed(seed)
    gen = ScenarioGenerator(cf.scenario)
    vols = gen(cf.num_vol)
    obj = get_obj_multiple_runway(vols, separation_manner, cf.penalty)
    def f(x):
        x = decode(x)
        cost = obj(x)
        return cost
    sep = make_separation_matrix(vols, separation_manner)
    return vols, f, sep

# Algorithms
## 1. Swap-order
Make pairs of vol.
Assign decision variables to each pair qui decide the pair is whether swapped or not.


## 2. Swap-queue
Assign decision variables to each vol qui decide the assigned runway of vol is whether swapped of not.


In [96]:
def make_separation_matrix(vols: Scenario, sep: Separation = recat.TBS):
    S = np.asarray([
        [
            sep(vol1, vol2)
            for vol2 in vols
        ] for vol1 in vols
    ])
    S *= 1 - np.eye(cf.num_vol).astype(int)
    return S

In [97]:
def make_pair(vols: Scenario, shift: bool = False):
    pairs = np.asarray([[2 * k, 2 * k  + 1] for k in range(len(vols) // 2)])
    if shift:
        pairs = pairs + 1
        if pairs[-1, 1] == len(vols):
            pairs = pairs[:-1, ...]
    return pairs

In [142]:
def neg(x):
    return 1- x

In [406]:
def X(q):
    return np.array([q, neg(q)])

def retrieve_S(S, pj, pk):
    return S[pj, ...][..., pk]

In [407]:
def make_X_builder(qs):
    def f(k):
        return np.array([
            [qs[k]],
            [1 - qs[k]]
        ])
    return f


def cost(pairs, S):
    qs = np.asarray([dimod.Binary(f"q{k}") for k in range(len(pairs))])
    X = make_X_builder(qs)

    Lks = []

    for k, pk in enumerate(pairs):
        Xk = X(k)
        notXk = neg(Xk)
        Skk = retrieve_S(S, pk, pk)
        Lk_ego = Xk.T @ Skk @ notXk
        # Lk_ego = Skk * np.outer(Xk, notXk)

        if k == 0:
            Lks.append(Lk_ego)
            continue

        j = k - 1
        pj = pairs[j]
        Xj = X(j)
        Sjk = retrieve_S(S, pj, pk)
        Lk_inter = Xj.T @ Sjk @ notXk

        Lks.append(Lk_inter + Lk_ego)

    return np.sum(Lks)

In [408]:
def build_model(vols, seps, xs, shift: bool = False):
    print("xs:", xs)
    S = seps[xs, ...][..., xs]
    pairs = make_pair(vols, shift=shift)

    # build CQM
    cqm = dimod.ConstrainedQuadraticModel()
    cqm.set_objective(cost(pairs, S))

    # convert to BQM
    bqm, invert = dimod.cqm_to_bqm(cqm)

    return bqm, pairs

In [409]:
from collections import namedtuple
Sample = namedtuple("Sample", ["sample", "energy", "num_occurrences"])

def solve_bqm(bqm, num_reads: int = 100):
    sampler = SQASampler()
    sampleset = sampler.sample(bqm, num_reads=num_reads)
    first = sampleset.first
    return Sample(
        sample=np.asarray(list(first.sample.values())),
        energy=first.energy,
        num_occurrences=first.num_occurrences
    )

In [410]:
def initial_sol(cf):
    nx = cf.num_vol
    nr = cf.num_runway
    temp_assign = np.concatenate([np.arange(nr) for _ in range(nx // nr + 1)])
    return np.vstack([
        np.arange(nx),
        temp_assign[:nx]
    ])

In [411]:
def swap_xs(xs, pairs, sample):
    sampled_indices = pairs.copy()
    sampled_indices[sample.sample == 1, ...] = pairs[sample.sample == 1, ...][..., ::-1]
    sampled_indices = sampled_indices.flatten()
    if len(sampled_indices) < xs.shape[1]:
        original_indices = xs[0, ...].copy()
        original_indices[1:len(pairs)*2+1] = sampled_indices
        sampled_indices = original_indices
    return xs[..., sampled_indices]


In [431]:
# generate problem
cf = get_default_config()
vols, obj, sep = make_scenario(cf)

In [432]:
# initial sample
xc = initial_sol(cf)
ec = obj(xc)

In [433]:
for i in range(10):
    print("=+"*30)
    bqm, pairs = build_model(vols, sep, xc[0, ...], shift=bool(i % 2))
    sample = solve_bqm(bqm)
    xp = swap_xs(xc, pairs, sample)
    ep = obj(xp)

    if ep <= ec:
        xc = xp
        ec = ep
    print(f"[info] round:{i+1} cost:{ep}/best:{ec}  sample:{xp[0, ...]} pairs:{pairs} sample:{sample.sample}")

=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+
xs: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]
[info] round:1 cost:225.06666666666666/best:225.06666666666666  sample:[ 0  1  2  3  5  4  7  6  9  8 10 11 12 13 15 14 17 16 19 18 21 20 22 23
 24 25 26 27 28 29] pairs:[[ 0  1]
 [ 2  3]
 [ 4  5]
 [ 6  7]
 [ 8  9]
 [10 11]
 [12 13]
 [14 15]
 [16 17]
 [18 19]
 [20 21]
 [22 23]
 [24 25]
 [26 27]
 [28 29]] sample:[0 0 1 1 1 0 0 1 1 1 1 0 0 0 0]
=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+
xs: [ 0  1  2  3  5  4  7  6  9  8 10 11 12 13 15 14 17 16 19 18 21 20 22 23
 24 25 26 27 28 29]
[info] round:2 cost:448.3333333333333/best:225.06666666666666  sample:[ 0  2  1  5  3  4  7  9  6  8 10 12 11 15 13 17 14 19 16 21 18 20 22 23
 24 25 26 27 28 29] pairs:[[ 1  2]
 [ 3  4]
 [ 5  6]
 [ 7  8]
 [ 9 10]
 [11 12]
 [13 14]
 [15 16]
 [17 18]
 [19 20]
 [21 22]
 [23 24]
 [25 26]
 [27 28]] sample:[1 1 0 1 0 1 1 1 1 1 0 0 0 0]
=+=+=+

In [372]:
vols.to_dataframe()

,code,ready,due,category,operation
0,VOL0001,11,131,F,A
1,VOL0002,23,143,D,A
2,VOL0003,49,169,C,A
3,VOL0004,108,228,D,A
4,VOL0005,177,297,C,D
5,VOL0006,190,310,C,D


In [373]:
sep

array([[  0,  82,  82,  82,  82,  82],
       [148,   0,  74,  74,  75,  75],
       [180,  90,   0,  90,  90,  90],
       [148,  74,  74,   0,  75,  75],
       [180,  90,  90,  90,   0,  90],
       [180,  90,  90,  90,  90,   0]])

In [343]:
bqm, pairs = build_model(vols, sep, xc[0, ...])

In [344]:
sample = solve_bqm(bqm)

In [347]:
sampled_indices = pairs.copy()
sampled_indices[sample.sample == 1, ...] = pairs[sample.sample == 1, ...][..., ::-1]
sampled_indices.flatten()

array([0, 1, 2, 3, 5, 4])

In [348]:
xp = xc[..., sampled_indices.flatten()]
xp

array([[0, 1, 2, 3, 5, 4],
       [0, 1, 0, 1, 1, 0]])

In [351]:
ep = obj(xp)
ep

(131.0, 131.0)

In [320]:
xp = xc.copy()
print(xp)
print(sample.sample)
xp[sample.sample]


[[0 1 2 3 4 5]
 [0 1 0 1 0 1]]
[0 0 1]


array([[0, 1, 2, 3, 4, 5],
       [0, 1, 2, 3, 4, 5],
       [0, 1, 0, 1, 0, 1]])

In [352]:
bool(1)


True